In [124]:
import pandas as pd
import nltk
import numpy as np
from queue import *
import re

In [125]:
noticias = pd.read_csv('estadao_noticias_eleicao.csv')
noticias.set_index("idNoticia", inplace=True)

gabarito = pd.read_csv('gabarito.csv')

In [126]:
#Lê e separa as palavras de um texto de determinada notícia, para adicionar ou atualizar o índice 'palavra' do dicionário
def cataloga_palavras(dicionario, texto, idNoticia):
    palavras = nltk.word_tokenize(texto)
    
    for palavra in palavras:
        palavra = palavra.lower()   #o algoritmo não é case-sensitive
        if palavra in dicionario:
            if idNoticia in dicionario[palavra]:
                dicionario[palavra][idNoticia] += 1
            else:
                dicionario[palavra][idNoticia] = 1
        else:
            dicionario[palavra] = {}
            dicionario[palavra][idNoticia] = 1

In [127]:
#Retorna os ids de notícias que possuem aquela palavra
def indice_invertido(palavra):
    return dicionario[palavra]

In [128]:
dicionario = {}

#Cataloga as palavras de todas as notícias da tabela
for index, row in noticias.iterrows():
    
    #Tratamento de valores NaN na tabela
    titulo = '' if str(row['titulo']) == 'nan' else str(row['titulo'])
    subTitulo = '' if str(row['subTitulo']) == 'nan' else str(row['subTitulo'])
    conteudo = '' if str(row['conteudo']) == 'nan' else str(row['conteudo'])
    
    texto = titulo + " " + subTitulo + " " + conteudo
    
    cataloga_palavras(dicionario, texto, index)

In [129]:
#Calcula IDF para um indice invertido
def calcula_idf(indice):
    m = noticias.shape[0]
    k = len(indice)
    return np.log((m + 1)/k) 

In [130]:
#Busca binaria, retorna os 5 documentos mais relevantes em que os termos da consulta estão presentes.
def binaria(consulta):
    termos = consulta.split()
    buscas = PriorityQueue()
    for termo in termos:
        termo = termo.lower()
        indice = indice_invertido(termo)
        buscas.put((len(indice), indice.keys()))  #A fila de prioridades armazena a quantidade de documentos em que
                                                  #cada termo aparece e os índices desses documentos
    if len(termos) == 1:
        return buscas.get()[1]
    while buscas.qsize()> 1:
        parte1 = set(buscas.get()[1])
        parte2 = set(buscas.get()[1])
        parcial = parte1.intersection(parte2)  #Interseção de dois itens da fila, até que só exista um elemento
        size = len(parcial)
        buscas.put((size, list(parcial)))
    return buscas.get()[1][0:5]

In [131]:
def intersect_tf(s1, s2):
    result = {}
    docs = set(s1.keys()).intersection(set(s2.keys()))
    for key in docs:
        result[key] = s1[key] + s2[key]  #Soma do tf no documento
    return result

def tf(consulta):
    termos = consulta.split()
    buscas = PriorityQueue()
    for termo in termos:
        termo = termo.lower()
        indice = indice_invertido(termo)
        buscas.put((len(indice), indice))  #A fila de prioridades armazena a quantidade de documentos em que
                                           #cada termo aparece e o índice invertido de cada termo
    if len(termos) == 1:
        return buscas.get()[1]
    while buscas.qsize()> 1:
        parte1 = buscas.get()[1]
        parte2 = buscas.get()[1]
        parcial = intersect_tf(parte1, parte2)  #Interseção de dois itens da fila, até que só exista um elemento
        size = len(parcial)
        buscas.put((size, parcial))
    return ordena_top5(buscas)

In [132]:
def intersect_tfidf(s1, s2):
    result = {}
    docs = set(s1.keys()).intersection(set(s2.keys()))
    for key in docs:
        result[key] = s1[key] * calcula_idf(s1) + s2[key] * calcula_idf(s2)  #Soma do idf no documento
    return result

def tfidf(consulta):
    termos = consulta.split()
    buscas = PriorityQueue()
    for termo in termos:
        termo = termo.lower()
        indice = indice_invertido(termo)
        buscas.put((len(indice), indice))  #A fila de prioridades armazena a quantidade de documentos em que
                                           #cada termo aparece e o índice invertido de cada termo
    if len(termos) == 1:
        return buscas.get()[1]
    while buscas.qsize()> 1:
        parte1 = buscas.get()[1]
        parte2 = buscas.get()[1]
        parcial = intersect_tfidf(parte1, parte2)  #Interseção de dois itens da fila, até que só exista um elemento
        size = len(parcial)
        buscas.put((size, parcial))
    return ordena_top5(buscas)

In [133]:
def intersect_bm25(s1, s2):
    k = np.random.uniform(low=1.2, high=2)
    result = {}
    docs = set(s1.keys()).intersection(set(s2.keys()))
    for key in docs:
        #Cálculo do bm25
        result[key] = (calcula_idf(s1) * s1[key] * (k + 1) / (s1[key] + k)) + (calcula_idf(s2) * s2[key] * (k + 1) / (s2[key] + k))
    return result

def bm25(consulta):
    termos = consulta.split()
    buscas = PriorityQueue()
    for termo in termos:
        termo = termo.lower()
        indice = indice_invertido(termo)
        buscas.put((len(indice), indice))  #A fila de prioridades armazena a quantidade de documentos em que
                                           #cada termo aparece e o índice invertido de cada termo
    if len(termos) == 1:
        return buscas.get()[1]
    while buscas.qsize()> 1:
        parte1 = buscas.get()[1]
        parte2 = buscas.get()[1]
        parcial = intersect_bm25(parte1, parte2)  #Interseção de dois itens da fila, até que só exista um elemento
        size = len(parcial)
        buscas.put((size, parcial))
    return ordena_top5(buscas)

In [134]:
def ordena_top5(buscas):
    sorted_result = sorted(buscas.get()[1].items(), key=lambda x: -x[1])  #Documentos em ordem decrescente de idf
    top5 = sorted_result[0:5]  #Apenas os 5 primeiros resultados
    top5 = [x[0] for x in top5]  #Listar pelo id do documento
    return top5

In [135]:
d = {'str_busca': ["segundo turno", "lava jato", "projeto de lei", "compra de voto", "minstério público"],
     'busca_binaria': [binaria("segundo turno"), binaria("lava jato"), binaria("projeto de lei"), binaria("compra de voto"), binaria("ministério público")], 
     'tf': [tf("segundo turno"), tf("lava jato"), tf("projeto de lei"), tf("compra de voto"), tf("ministério público")],
     'tfidf': [tfidf("segundo turno"), tfidf("lava jato"), tfidf("projeto de lei"), tfidf("compra de voto"), tfidf("ministério público")],
     'bm25': [bm25("segundo turno"), bm25("lava jato"), bm25("projeto de lei"), bm25("compra de voto"), bm25("ministério público")]}
resposta = pd.DataFrame(data=d)
resposta = resposta[['str_busca', 'busca_binaria', 'tf', 'tfidf', 'bm25']]
resposta

,str_busca,busca_binaria,tf,tfidf,bm25
0,segundo turno,"[2048, 1, 2049, 2050, 4096]","[2744, 7, 2112, 7672, 2388]","[2744, 2112, 7672, 1235, 2388]","[2744, 2112, 7672, 2388, 2178]"
1,lava jato,"[3, 13, 15, 27, 6177]","[163, 353, 2807, 127, 359]","[163, 353, 2807, 127, 359]","[163, 353, 2807, 127, 359]"
2,projeto de lei,"[3584, 6145, 8194, 8706, 6660]","[7, 3942, 7017, 1250, 6942]","[7017, 2853, 2232, 3171, 6461]","[2853, 3171, 2232, 6699, 6461]"
3,compra de voto,"[7424, 2178, 6531, 5122, 2311]","[3942, 7017, 5129, 2047, 748]","[7343, 2047, 7293, 6791, 7017]","[2200, 2047, 2178, 7343, 7293]"
4,minstério público,"[8194, 7, 4104, 8201, 4109]","[6798, 8018, 6244, 6965, 6550]","[6798, 8018, 6244, 6965, 6550]","[6798, 8018, 6244, 6965, 6550]"


In [136]:
def apk(actual, predicted, k):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [164]:
#Tratando o formato das colunas
def get_expected_results(df):   
    expected_answers = []
    for query_result in df:        
        as_str = re.sub('[,\[\]]', '', query_result)
        as_list = as_str.split(" ")
        list_of_int = list(map(int, as_list))
        expected_answers.append(list_of_int)
    return expected_answers

In [165]:
print(mapk(get_expected_results(gabarito.busca_binaria), resposta.busca_binaria, k=5))
print(mapk(get_expected_results(gabarito.tf), resposta.tf, k=5))
print(mapk(get_expected_results(gabarito.tfidf), resposta.tf, k=5))
print(mapk(get_expected_results(gabarito.bm25), resposta.bm25, k=5))

print(mapk(get_expected_results(gabarito.google), resposta.busca_binaria, k=5))
print(mapk(get_expected_results(gabarito.google), resposta.tf, k=5))
print(mapk(get_expected_results(gabarito.google), resposta.tfidf, k=5))
print(mapk(get_expected_results(gabarito.google), resposta.bm25, k=5))

1.0
1.0
0.608666666667
0.778
0.0
0.048
0.088
0.168


Em relação ao gabarito, os modelos mais precisos foram a Busca Binária e TF.
As comparações com os resultados do google tiveram precisão baixa, como esperado. Neste caso, o modelo BM25 obteve melhores resultados.